In [ ]:

from time import time_ns
import pysoem

class AM8111Profile:

    MODE_CSP                = 8     # cyclic synchronous position
    MODE_CSV                = 9     # cyclic synchronous velocity
    MODE_CST                = 10    # cyclic synchronous torque
    MODE_CSTCA              = 11    # cyclic synchronous torque with commutation angle

    # status word
    NOT_READY_TO_SWITCH_ON  = 0     # xxx0 xxxx x0xx 0000
    READY_TO_SWITCH_ON      = 1     # xxx0 xxxx x01x 0001
    SWITCHED_ON             = 2     # xxx0 xxxx x1xx 0011
    OPERATION_ENABLED       = 4     # xxx0 xxxx x1xx 0111
    FAULT                   = 8
    
    QUICK_STOP              = 32
    SWITCH_ON_DISABLED      = 64    # xxx0 xxxx x1xx 0000
    WARNING                 = 128

    # control word LoByte
    FAULT_RESET             = '10000000'    # 1xxx xxxx 15
    SHUTDOWN                = '00000110'    # 0xxx x110 2,6,8
    SWITCH_ON               = '00000111'    # 0xxx 0111 3,5
    ENABLE_OPERATION        = '00001111'    # 0xx0 1111 4
    
    """
    status                                              control

    xxx0 xxxx x0xx 0000     not ready to switch on
               |                                        
    xxx0 xxxx x1xx 0000     switch on disabled
                      |                                 0000 0000 0xxx x110       shutdown
    xxx0 xxxx x01x 0001     ready to switch on
                     |                                  0000 0000 0xxx x111       switch on
    xxx0 xxxx x1xx 0011     switched on
                    |                                   0000 0000 0xx0 1111       enable operation
    xxx0 xxxx x1xx 0111     operation enabled


    xxx0 xxxx x0xx 1111     fault reaction active
    xxx0 xxxx x0xx 1000     fault                       0000 0000 1xxx xxxx       fault reset

    
                                                        0000 0000 0xxx xx0x       disable voltage
                                                        0000 0000 0xxx x01x       
    """
    
    status = [
        NOT_READY_TO_SWITCH_ON, READY_TO_SWITCH_ON, SWITCHED_ON, OPERATION_ENABLED, 
        FAULT, QUICK_STOP, SWITCH_ON_DISABLED, WARNING]
    
    status_name = [
        'NOT_READ_TO_SWITCH_ON','READY_TO_SWITCH_ON', 'SWITCHED_ON', 'OPERATION_ENABLED', 
        'FAULT', 'QUICK_STOP', 'SWITCH_ON_DISABLED', 'WARNING']
    
    @staticmethod
    def __str__(state):
        return ",".join([f"{AM8111Profile.status_name[i]}" 
                        for i,s in enumerate(AM8111Profile.status) 
                            if ((s & state) == s) and (s not in [0])
                        ])
    @staticmethod
    def __get__(state):
        return [s for s in AM8111Profile.status if ((s & state) == s) and (s not in [0])]

    @staticmethod
    def __has__(value, state):
        return (state | value) == value

    @staticmethod
    def __transit__(value):

        # transition, state, index

        if AM8111Profile.__has__(value, AM8111Profile.FAULT):
            return [
                (AM8111Profile.FAULT_RESET, AM8111Profile.SWITCH_ON_DISABLED, 15)
                ]

        if AM8111Profile.__has__(value, AM8111Profile.OPERATION_ENABLED):
            return [
                (AM8111Profile.SWITCH_ON, AM8111Profile.SWITCHED_ON, 5), 
                (AM8111Profile.SHUTDOWN, AM8111Profile.READY_TO_SWITCH_ON, 8)
                ]
        
        if AM8111Profile.__has__(value, AM8111Profile.SWITCHED_ON):
            return [                
                (AM8111Profile.ENABLE_OPERATION, AM8111Profile.OPERATION_ENABLED, 4),
                (AM8111Profile.SHUTDOWN, AM8111Profile.READY_TO_SWITCH_ON, 6)
                ]
        
        if AM8111Profile.__has__(value, AM8111Profile.SWITCHED_ON):
            return [
                (AM8111Profile.ENABLE_OPERATION, AM8111Profile.OPERATION_ENABLED, 4)
                ]   
        
        if AM8111Profile.__has__(value, AM8111Profile.READY_TO_SWITCH_ON):
            return [
                (AM8111Profile.SWITCH_ON, AM8111Profile.SWITCHED_ON, 3),                
                ]

        if AM8111Profile.__has__(value, AM8111Profile.SWITCH_ON_DISABLED):
            return [
                (AM8111Profile.SHUTDOWN, AM8111Profile.READY_TO_SWITCH_ON, 2)
                ]    

        return []    
    
    warning_name = [None, None, 'UNDER_VOLTAGE', 'OVER_VOLTAGE', 'OVER_TEMPERATURE', 
                    'I2T_AMPLIFIER', 'I2T_MOTOR', 'ENCODER']
    error_name = ['ADC_ERROR', 'OVER_CURRENT', 'UNDER_VOLTAGE', 'OVER_VOLTAGE', 'OVER_TEMPERATURE', 
                  'I2T_AMPLIFIER', 'I2T_MOTOR', 'ENCODER', 'WATCHDOG']

    @staticmethod
    def __info__(value, mode='e'):
        value = bin(value)[2:].zfill(16)[::-1]
        value = list(map(lambda x: int(x), value))
        if mode == 'w':            
            return ",".join([f"{AM8111Profile.warning_name[i]}" 
                        for i,n in enumerate(AM8111Profile.warning_name) 
                            if n is not None and value[i] == 1
                        ])
        if mode == 'e':            
            return ",".join([f"{AM8111Profile.error_name[i]}" 
                        for i,n in enumerate(AM8111Profile.error_name) 
                            if n is not None and value[i] == 1
                        ])
        return ""

    
#value = int('0000010000100001',2)
#trans = AM8111Profile.__transit__(value)
#print(f"{time_ns()} {trans}")

AM8111Profile.__info__(256, mode='e')


'Watchdog'

In [8]:
import numpy as np

GEAR_BOX_RATIO = 1000. * 1.75
SPINDLE_PITCH = 5.                                          # mm
TIMINGBELT_TRANSMISSION_GEAR_RATIO = 2.                
MOTOR_INCREMENTAL_POSITIONS = 268_435 # 8_388_608           # inc/s
CYLINDER_DIAMETER = 15.                                     # mm
CYLINDER_AREA = CYLINDER_DIAMETER ** 2 * np.pi / 4.         # mm^2

def incs2mulmin(value):
    # mm/r
    transmission = SPINDLE_PITCH / (TIMINGBELT_TRANSMISSION_GEAR_RATIO * GEAR_BOX_RATIO)
    # µl/r
    injectionRateRotation = transmission * CYLINDER_AREA    # mm^2 ~ µl
    # µl/inc
    injectionRateIncrement = injectionRateRotation / MOTOR_INCREMENTAL_POSITIONS
    print(f"{injectionRateRotation} {injectionRateIncrement}")    
    return value * injectionRateIncrement * 60
    
def mulmin2incs(value):
    # inc/s; 838_860_800 max
    # mm/r
    transmission = SPINDLE_PITCH / (TIMINGBELT_TRANSMISSION_GEAR_RATIO * GEAR_BOX_RATIO)
    # µl/r
    injectionRateRotation = transmission * CYLINDER_AREA    # mm^2 ~ µl
    # µl/inc
    injectionRateIncrement = injectionRateRotation / MOTOR_INCREMENTAL_POSITIONS
    return value / (injectionRateIncrement * 60)

mulmin2incs(2388), incs2mulmin(24_185_993)

0.2524494096634655 9.404489342427982e-07


(42320213.83707021, 1364.7414804272266)